In [205]:
import pandas as pd
import numpy as np
import random

In [206]:
random.seed(10)
np.random.seed(10)

In [207]:
def createParticle(particleSize, x_min = 0, x_max = 4):
    particle = x_min + (x_max - x_min) * np.random.uniform(low = 0, high = 1, size = particleSize)
    return particle

In [208]:
createParticle(10)

array([3.08528257, 0.0830078 , 2.53459294, 2.99521553, 1.99402805,
       0.89918658, 0.79225146, 3.04212285, 0.67644335, 0.35335926])

In [209]:
def createParticleVelocities(particleSize, v_min = -4, v_max = 4):
    velocities = v_min + (v_max - v_min) * np.random.uniform(low = 0, high = 1, size = particleSize)
    return velocities

In [210]:
createParticleVelocities(10)

array([ 1.48287855,  3.62714677, -3.96841387,  0.09753811,  2.50096769,
        0.90020853,  1.77404254, -1.66499145,  3.34219298,  1.71660627])

In [211]:
def createPopParticles(popSize, particle_length):
    pop = [createParticle(particle_length) for i in range(0,popSize)]
    return np.array(pop)

In [212]:
def createPopVelocities(popSize, particle_length):
    pop_vel = [createParticleVelocities(particle_length) for i in range(0,popSize)]
    return np.array(pop_vel)

In [213]:
def generateSantasPathFromParticlesPop(population):
    path_populations = np.argsort(population).tolist()
    #The constraint is that for each path should start and end at 0!
    for path in path_populations:
        path.remove(0)
        path.insert(0,0)
        path.insert(len(path), 0)
    return path_populations

In [214]:
import math
def is_prime(n):
    if n == 2:
        return True
    if n % 2 == 0 or n <= 1:
        return False

    sqr = int(math.sqrt(n)) + 1

    for divisor in range(3, sqr, 2):
        if n % divisor == 0:
            return False
    return True
def not_prime(n):
    if n == 2:
        return False
    if n % 2 == 0 or n <= 1:
        return True

    sqr = int(math.sqrt(n)) + 1

    for divisor in range(3, sqr, 2):
        if n % divisor == 0:
            return True
    return False

def cities_distance(route, coors):
    df = coors.copy()
    df = df.loc[route].reset_index()
    df = df.rename(columns = {'index' : 'CityId'})
    df['dist'] = np.sqrt((df.X - df.X.shift())**2 + (df.Y - df.Y.shift())**2)
    df = df.drop(0)
    idx = (df.index % 10 == 0)
    idx = df.loc[idx].CityId.apply(not_prime)
    idx = idx.index[idx.values]
    df.loc[idx, 'dist'] = df.loc[idx, 'dist'] + df.loc[idx, 'dist'] / 10
    return np.sum(df['dist'])

def euclidean_dist(id_a, id_b, coors):
    a_coordinates = coors[id_a]
    b_coordinates = coors[id_b]
    partial_sum_X = np.power(a_coordinates[0] - b_coordinates[0], 2)
    partial_sum_Y = np.power(a_coordinates[1] - b_coordinates[1], 2)
    distance = np.sqrt(partial_sum_X + partial_sum_Y)
    return(distance)

In [215]:
def evaluate_paths(paths, cities):
    distances = []
    for path in paths:
        distances.append(cities_distance(path, cities))
    return np.array(distances)

In [216]:
def swapMutation(pop):
    pos_to_swap = np.random.choice(range(0,len(pop)), size = 2, replace = False)
    pop[pos_to_swap[0]], pop[pos_to_swap[1]] = pop[pos_to_swap[1]], pop[pos_to_swap[0]]
    return pop

In [223]:
def particleSwarmOptimization(path_length,
                              coors,
                              ro=50,
                              number_of_iterations = 100,
                              decrement_factor = 0.975,
                              inertia_weight = 0.9,
                              wait_interval = 20):

    cities = coors
    #iteration counter
    t = 0
    #Initialization
    pop = createPopParticles(popSize= ro, particle_length=path_length)
    best_pop = pop
    vel = createPopVelocities(popSize= ro, particle_length=path_length)
    paths = generateSantasPathFromParticlesPop(pop)
    distances = evaluate_paths(paths, cities)
    best_distances = distances
    #Initialize support variables where best results will be stored
    best_iteration = t
    best_particle_index = np.argsort(distances)[0]
    best_particle = pop[best_particle_index]
    global_min_distance = distances[best_particle_index]
    c1 = 2 #cognitive param
    c2 = 2 #social param
    r1 = np.random.uniform(low = 0, high = 1)
    r2 = np.random.uniform(low = 0, high = 1)
    
    result = [[t,global_min_distance]]
    for t in range(1,number_of_iterations + 1):
        print(t)
        inertia_weight = inertia_weight * decrement_factor
        if inertia_weight < 0.4:
            inertia_weight = 0.4
        vel = inertia_weight * vel
        vel = vel +  c1 * r1 * (best_pop - pop)
        vel =  vel + c2 * r2 * (best_particle - pop)
        pop = pop + vel
        elements_to_mutate = np.random.choice(range(0,ro), size = round(ro/10), replace=False)
        #print(elements_to_mutate)
        for el in elements_to_mutate:            
            pop[el] = swapMutation(pop[el])
        paths = generateSantasPathFromParticlesPop(pop)
        distances = evaluate_paths(paths, cities)
        distance_comparison = distances < best_distances
        best_distances[distance_comparison] = distances[distance_comparison]
        
        #Update global best results if necessary
        current_iteration_best_index = np.argsort(distances)[0]
        current_min_distance = distances[current_iteration_best_index]
        if current_min_distance < global_min_distance:
            best_iteration = t
            best_particle = pop[current_iteration_best_index]
            global_min_distance = current_min_distance
        elif t > best_iteration + wait_interval:
            break
        result.append([t,global_min_distance]) 
    return result

In [224]:
cities = pd.read_csv('cities.csv')
coors = cities[['X', 'Y']]

In [225]:
res = particleSwarmOptimization(len(coors)-1,
                              coors)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [226]:
print(res)

[[0, 446336639.818132], [1, 445960580.6894744], [2, 445926110.4559802], [3, 445926110.4559802], [4, 445926110.4559802], [5, 445926110.4559802], [6, 445926110.4559802], [7, 445926110.4559802], [8, 445391191.9969069], [9, 445391191.9969069], [10, 445391191.9969069], [11, 445391191.9969069], [12, 445391191.9969069], [13, 445391191.9969069], [14, 445391191.9969069], [15, 445391191.9969069], [16, 445391191.9969069], [17, 445391191.9969069], [18, 445391191.9969069], [19, 445391191.9969069], [20, 445391191.9969069], [21, 445391191.9969069], [22, 445391191.9969069], [23, 445391191.9969069], [24, 445391191.9969069], [25, 445391191.9969069], [26, 445391191.9969069], [27, 445391191.9969069], [28, 445391191.9969069]]
